In [38]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import os
from PIL import Image
from numpy import *

In [39]:
cwd = os.getcwd()
loadpath = cwd + "/data/custom_data.npz"
l = np.load(loadpath)

# See what's in here
print (l.files)

# Parse data
trainimg = l['trainimg']
trainlabel = l['trainlabel']
testimg = l['testimg']
testlabel = l['testlabel']
imgsize = l['imgsize']
use_gray = l['use_gray']
ntrain = trainimg.shape[0]
nclass = trainlabel.shape[1]
dim    = trainimg.shape[1]
ntest  = testimg.shape[0]
print ("%d train images loaded" % (ntrain))
print ("%d test images loaded" % (ntest))
print ("%d dimensional input" % (dim))
print ("Image size is %s" % (imgsize))
print ("%d classes" % (nclass))

['trainlabel', 'imgsize', 'trainimg', 'testimg', 'testlabel', 'use_gray']
15000 train images loaded
10000 test images loaded
784 dimensional input
Image size is [28 28]
2 classes


In [40]:
n_classes = 2
batch_size = 128

x = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.float32, shape=[None, 2])
keep_prob = tf.placeholder(tf.float32)

def conv2d(x,W):
    return tf.nn.conv2d(x,W, strides=[1,1,1,1], padding='SAME')

def maxpool2d(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')



def convolutional_neural_network(x):
    weights = {'W_conv1':tf.Variable(tf.random_normal([5,5,1,32])),
               'W_conv2':tf.Variable(tf.random_normal([5,5,32,64])),
             #  'W_conv3':tf.Variable(tf.random_normal([5,5,64,128])),
              # 'W_conv4':tf.Variable(tf.random_normal([5,5,128,256])),
               'W_fc':tf.Variable(tf.random_normal([7*7*64,1024])),
               'out':tf.Variable(tf.random_normal([1024,n_classes]))}
                      
    biases =  {'b_conv1':tf.Variable(tf.random_normal([32])),
               'b_conv2':tf.Variable(tf.random_normal([64])),
               #'b_conv3':tf.Variable(tf.random_normal([128])),
               #'b_conv4':tf.Variable(tf.random_normal([256])),
               'b_fc':tf.Variable(tf.random_normal([1024])),
               'out':tf.Variable(tf.random_normal([n_classes]))}
               
    x = tf.reshape(x, shape=[-1,112,112,1])
    
    conv1 = tf.nn.relu(conv2d(x,weights['W_conv1'])+biases['b_conv1'])
    conv1 = maxpool2d(conv1)
    
    conv2 = tf.nn.relu(conv2d(conv1,weights['W_conv2'])+biases['b_conv2'])
    conv2 = maxpool2d(conv2)

    #conv3 = tf.nn.relu(conv2d(conv2,weights['W_conv3'])+biases['b_conv3'])
    # conv3 = maxpool2d(conv3)

    #conv4 = tf.nn.relu(conv2d(conv3,weights['W_conv4'])+biases['b_conv4'])
    #conv4 = maxpool2d(conv4)
    
    fc = tf.reshape(conv2, [-1,7*7*64])
    fc = tf.nn.relu(tf.matmul(fc,weights['W_fc'])+biases['b_fc'])
    
    h_fc1_drop = tf.nn.dropout(fc, keep_prob)

    output = tf.matmul(h_fc1_drop , weights['out'])+biases['out']
    
    return output


In [41]:
def train_neural_network(x):
    prediction = convolutional_neural_network(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)
        
    hm_epochs = 10
    
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        
        for epoch in range(hm_epochs):
            epoch_loss = 0
            num_batch = int(ntrain/batch_size)+1
            for _ in range(num_batch):
                randidx = np.random.randint(ntrain, size=batch_size)
                epoch_x,epoch_y = trainimg[randidx, :],trainlabel[randidx, :]
                _,c = sess.run([optimizer,cost], feed_dict = {x:epoch_x,y:epoch_y,keep_prob: 0.5})
                epoch_loss+=c
            print('Epoch', epoch, 'completed out of', hm_epochs,'loss',epoch_loss)
                
        correct = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
        print('Accuracy:', accuracy.eval({x:testimg, y :testlabel,keep_prob: 1.0}))

In [42]:
train_neural_network(x)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
('Epoch', 0, 'completed out of', 10, 'loss', 3434299.7177734375)
('Epoch', 1, 'completed out of', 10, 'loss', 1036310.2841796875)
('Epoch', 2, 'completed out of', 10, 'loss', 527248.65283203125)
('Epoch', 3, 'completed out of', 10, 'loss', 322246.65478515625)
('Epoch', 4, 'completed out of', 10, 'loss', 209481.68200683594)
('Epoch', 5, 'completed out of', 10, 'loss', 148878.95721435547)
('Epoch', 6, 'completed out of', 10, 'loss', 79997.593414306641)
('Epoch', 7, 'completed out of', 10, 'loss', 37924.544906616211)
('Epoch', 8, 'completed out of', 10, 'loss', 23115.324295043945)
('Epoch', 9, 'completed out of', 10, 'loss', 14987.26305770874)
('Accuracy:', 0.52770001)
